In [20]:
import rioxarray
import xarray as xr
import numpy as np
import rasterio
import rio_cogeo.cogeo
import matplotlib.pyplot as plt
import s3fs 
from pyproj import CRS

In [ ]:
def goes_to_wgs(ds, variable_name):

    # convert coordinates to meters https://gis.stackexchange.com/a/350006
    sat_height = ds["goes_imager_projection"].attrs["perspective_point_height"]
    ds = ds.assign_coords({
        "x": ds["x"].values * sat_height,
        "y": ds["y"].values * sat_height,
    })
    
    crs = CRS.from_cf(ds["goes_imager_projection"].attrs)
    ds.rio.write_crs(crs.to_string(), inplace=True)
    
    da = ds[variable_name]

    return da.rio.reproject("epsg:4326")

In [ ]:
#From AWS
fs = s3fs.S3FileSystem(anon=True)

path = "s3://noaa-goes16/ABI-L1b-RadF/2017/129/11/OR_ABI-L1b-RadF-M3C03_G16_s20171291115392_e20171291126159_c20171291126196.nc"

with fs.open(path) as fileObj:
    with xr.open_dataset(fileObj, engine='h5netcdf') as ds:

        da = goes_to_wgs(ds, variable_name="Rad")
    
        COG_PROFILE = {"driver": "COG", "compress": "DEFLATE", "predictor": 2}
        da.rio.to_raster("goes16.tif", **COG_PROFILE)

In [ ]:
#From downloaded file
path = "/Users/sidchaudhary/Downloads/OR_ABI-L1b-RadF-M6C01_G16_s20240010010205_e20240010019513_c20240010019563.nc"

with xr.open_dataset(path, engine='h5netcdf') as ds:

    da = goes_to_wgs(ds, variable_name="Rad")

    COG_PROFILE = {"driver": "COG", "compress": "DEFLATE", "predictor": 2}
    da.rio.to_raster("goes16.tif", **COG_PROFILE)


In [ ]:
# Path to the Cloud Optimized GeoTIFF (COG)
cog_filename = "goes16.tif"

# Open the COG file using rasterio
with rasterio.open(cog_filename) as src:
    # Read the data from the COG
    data = src.read(1)  # Reading the first band (radiance data)
    
    # Get the bounds (left, bottom, right, top) of the dataset in geographic coordinates
    bounds = src.bounds
    extent = [bounds.left, bounds.right, bounds.bottom, bounds.top]
    
    # Set up a figure
    plt.figure(figsize=(10, 6))
    
    # Plot the data using imshow and set the extent to geographic bounds
    plt.imshow(data, cmap='turbo', extent=extent, origin='upper')
    
    # Add labels and title
    plt.xlabel('Longitude')
    plt.ylabel('Latitude')
    plt.title('GOES-16 Radiance in Geographic Coordinates')

    # Add a colorbar
    plt.colorbar(label="Radiance")

    # Display the plot
    plt.show()